In [ ]:
"""
인공 신경망 - 회귀, 분류작업 위해 수행

-퍼셉트론은 inputs/ weights/ transfer function/ activation function 으로 이뤄짐 
-Activation function(활성화 함수) -> 다음 뉴런으로 신호를 보낼지 말지, 보내면 어떤 신호를 보낼지 판단

모든 노드들이 1대1로 연결되어 있는 구조 ; Fully - connected layers 

Deep learning - hiddne layer가 2개 이상 있는 경우를 말한다. 
hiddne layer  -> input(특징)에 대해 추상화된 특징 -> hidden layer가 2개라 해서 성능이 무조건 좋아지는거 아님
w(가중치)가 에러에 얼마나 영향력을 미쳤는지에 따라 에러 값만큼 분배한다.
->output 기준 역방향으로 weight들 업데이트 한다 -> 이후 forward propogation(순전파)통해 학습하고 에러가 또 있으면 
다시 backward propogation(역전파) 진행 -> 에러 안나올때까지 계속해서 반복 : 1-batch-size(1건의 데이터를 읽을때마다 weight update일어남)
if 10-batch-size = 10개의 데이터를 한번에 읽어 w update진행

output layer 에서 시그모이드 함수 적용해 0 or 1로 나타낸다.


변수가 얼마만큼 중요하냐 나타내는게 가중치변수

"""

In [ ]:
"""
케라스(딥러닝 프레임웍)을 이용한 점수 에측
입력 특징: 공부시간, 출석일수, 과제제출횟수 
출력: 시험점수

??? 프레임웍 : framework -> 빠르고 안정적이고 쉽고 적은 비용으로 개발 할수 있도록 해주는 라이브러리 집합체
딥러닝 프레임웍 : 텐서 플로우(케라스), 파이토치, 까페

복잡한 종류의 데이터를 가지고 관계를 형성할수 있는 신경망 구성할수 있다

모든 bias,weight등은 random으로 설정 된다. 
training하는 이유 ->모델을 만들기 위해 -> 가중치를 조정하기 위해 
cost,loss값이 가장 작게 나오는 w와b를 찾는게 목표
최적의 w,b 찾는 방법으로는 경사하강법이 있다

"""

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
model=keras.Sequential([layers.Dense(units=1, input_shape=[3])])  #keras 안에 있는 sequential이용해모델 생성  , 
                                                                  #units => 출력차원 , input_shape => 입력차원
#입력 차원은 3개 이고 출력 차원은 1개임, 모델은 Sequential이다

In [3]:
model=keras.Sequential([ 
layers.Dense(units=4  ,input_shape=[2], activation='relu'),
layers.Dense(units=3  , activation='relu'),    #input shape은 위에 썼음으로 다시 안써도 됨
layers.Dense(units=1  , activation='sigmoid')

])

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 4)                 12        
                                                                 
 dense_2 (Dense)             (None, 3)                 15        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 31 (124.00 Byte)
Trainable params: 31 (124.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model.compile(optimizer='adam', loss= 'mae')

In [6]:
import pandas as pd

In [72]:
red_wine=pd.read_csv('red-wine.csv')

In [73]:
red_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [74]:
df_train=red_wine.sample(frac=0.7, random_state=20231031)   #표본 추출할때 사용 하는 함수 (구간 0.7로 나눔)

In [75]:
len(df_train)

1119

In [76]:
df_train.index

Int64Index([1385, 1504, 1269, 1457,  671,  404, 1327, 1226,  803, 1563,
            ...
            1301,  120,  199,   40, 1052, 1043, 1130,  363,  386, 1008],
           dtype='int64', length=1119)

In [77]:
df_test=red_wine.drop(df_train.index) #df_train의 인덱스 모두 제거

In [78]:
len(df_test)

480

In [79]:
df_train[:2]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1385,8.0,0.81,0.25,3.4,0.076,34.0,85.0,0.99668,3.19,0.42,9.2,5
1504,7.5,0.38,0.57,2.3,0.106,5.0,12.0,0.99605,3.36,0.55,11.4,6


In [80]:
max_=df_train.max()        # 각 열 단위 최댓값
min_=df_train.min()

In [81]:
#정규화 
df_train=(df_train-min_) / (max_-min_)
df_test=(df_test-min_) / (max_-min_)
#df_train-min_ #각 열 단위 최솟값 뺸값

In [82]:
df_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1385,0.309091,0.570248,0.25,0.171233,0.106845,0.464789,0.279152,0.485316,0.354331,0.053892,0.142857,0.4
1504,0.263636,0.214876,0.57,0.095890,0.156928,0.056338,0.021201,0.439060,0.488189,0.131737,0.535714,0.6
1269,0.081818,0.305785,0.03,0.061644,0.053422,0.380282,0.286219,0.053598,0.598425,0.293413,1.000000,1.0
1457,0.272727,0.305785,0.33,0.068493,0.103506,0.366197,0.279152,0.513216,0.527559,0.149701,0.107143,0.4
671,0.327273,0.504132,0.21,0.054795,0.103506,0.056338,0.024735,0.494126,0.362205,0.113772,0.196429,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...
1043,0.445455,0.223140,0.41,0.547945,0.095159,0.239437,0.116608,0.625551,0.433071,0.287425,0.446429,0.8
1130,0.409091,0.396694,0.00,0.068493,0.076795,0.056338,0.014134,0.560206,0.346457,0.179641,0.357143,0.6
363,0.718182,0.280992,0.63,0.075342,0.098497,0.070423,0.031802,0.640969,0.196850,0.323353,0.321429,0.4
386,0.290909,0.347107,0.26,0.075342,0.126878,0.309859,0.148410,0.589574,0.527559,0.245509,0.142857,0.6


In [83]:
df_train=df_train.drop('quality',axis=1)
df_test=df_test.drop('quality',axis=1)

In [84]:
y_train=red_wine['quality']
y_text=red_wine['quality']

In [85]:
df_train.index

Int64Index([1385, 1504, 1269, 1457,  671,  404, 1327, 1226,  803, 1563,
            ...
            1301,  120,  199,   40, 1052, 1043, 1130,  363,  386, 1008],
           dtype='int64', length=1119)

In [86]:
y_train=red_wine['quality'][df_train.index]
y_test=red_wine['quality'][df_test.index]

In [105]:
###범주형으로 바꿀경우 원핫 인코딩 진행###

In [99]:
from sklearn.preprocessing import OneHotEncoder

In [100]:
ohe=OneHotEncoder()

In [104]:
pd.get_dummies(y_test)

,3,4,5,6,7,8
1,0,0,1,0,0,0
9,0,0,1,0,0,0
10,0,0,1,0,0,0
16,0,0,0,0,1,0
17,0,0,1,0,0,0
...,...,...,...,...,...,...
1581,0,0,1,0,0,0
1582,0,0,1,0,0,0
1587,0,0,0,1,0,0
1588,0,0,0,1,0,0


In [102]:
y_test_ohe=ohe.fit_transform([y_test])

In [ ]:
y_test_ohe

In [87]:
# 품질 : 0~10까지 부여
# 범주형 : 원핫 인코딩

In [94]:
model=keras.Sequential([
    layers.Dense(512, activation='relu',input_shape=[11]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(11, activation='softmax')  #범주형으로 바꿀 경우 ->test 데이터가 원핫인코딩 처리가 되있어야함
    #layers.Dense(1)
])

In [95]:
model.compile(
    optimizer='adam',
    loss='mae',
)

In [96]:
1119/32

34.96875

In [97]:
model.fit(   #bath-size default = 32 -> 35/35가 35까지 bath-size를 다 수행했다 의미
df_train,y_train, epochs = 12
)
## 원핫 인코딩 진행 안하고 범주형으로 진행시 로스 안주는 오류 나옴

Epoch 1/12
35/35 [==============================] - 1s 5ms/step - loss: 5.5257
Epoch 2/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 3/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 4/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 5/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 6/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 7/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 8/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 9/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 10/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 11/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257
Epoch 12/12
35/35 [==============================] - 0s 5ms/step - loss: 5.5257


In [98]:
model.predict(df_test)

15/15 [==============================] - 0s 2ms/step


array([[0.08778593, 0.09040712, 0.0911248 , ..., 0.09069036, 0.0900613 ,
        0.09525409],
       [0.08658186, 0.09212185, 0.08967583, ..., 0.09152452, 0.09021956,
        0.09343765],
       [0.08825403, 0.09092365, 0.09109236, ..., 0.09035829, 0.08953193,
        0.09360378],
       ...,
       [0.0873817 , 0.09075689, 0.090546  , ..., 0.09091623, 0.08903287,
        0.09441981],
       [0.0873084 , 0.09269276, 0.08922037, ..., 0.09148637, 0.08948068,
        0.09524472],
       [0.08665615, 0.09026726, 0.09080666, ..., 0.0908773 , 0.08873098,
        0.09583155]], dtype=float32)

In [92]:
df_test

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1,0.290909,0.628099,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.250000
9,0.263636,0.314050,0.36,0.356164,0.098497,0.225352,0.339223,0.567548,0.480315,0.281437,0.375000
10,0.190909,0.380165,0.08,0.061644,0.141903,0.197183,0.208481,0.428047,0.425197,0.125749,0.142857
16,0.354545,0.132231,0.56,0.061644,0.133556,0.478873,0.342756,0.501468,0.440945,0.251497,0.375000
17,0.318182,0.363636,0.28,0.054795,0.594324,0.211268,0.176678,0.494126,0.291339,0.568862,0.160714
...,...,...,...,...,...,...,...,...,...,...,...
1581,0.145455,0.363636,0.09,0.054795,0.068447,0.323944,0.091873,0.290015,0.629921,0.161677,0.517857
1582,0.136364,0.491736,0.10,0.116438,0.068447,0.169014,0.074205,0.260646,0.653543,0.101796,0.625000
1587,0.109091,0.404959,0.11,0.061644,0.090150,0.239437,0.077739,0.349486,0.637795,0.197605,0.446429
1588,0.236364,0.446281,0.33,0.109589,0.093489,0.464789,0.339223,0.298825,0.417323,0.269461,0.785714


In [107]:
df=pd.read_csv('diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [111]:
x=df.iloc[:,0:8]
y=df.iloc[:,8]

In [112]:
model=Sequential()

In [113]:
model.add(Dense(12, input_shape=[8], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 마지막 활성화 함수는 sigmoid사용

In [114]:
model.summary()   # param = 8*12 + 12

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 12)                108       
                                                                 
 dense_17 (Dense)            (None, 8)                 104       
                                                                 
 dense_18 (Dense)            (None, 1)                 9         
                                                                 
Total params: 221 (884.00 Byte)
Trainable params: 221 (884.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [115]:
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics = ['accuracy'])

In [116]:
model.fit(x,y,epochs=10, batch_size=5)

Epoch 1/10
154/154 [==============================] - 1s 1ms/step - loss: 1.7293 - accuracy: 0.5833
Epoch 2/10
154/154 [==============================] - 0s 1ms/step - loss: 0.7376 - accuracy: 0.6419
Epoch 3/10
154/154 [==============================] - 0s 1ms/step - loss: 0.7028 - accuracy: 0.6641
Epoch 4/10
154/154 [==============================] - 0s 1ms/step - loss: 0.6694 - accuracy: 0.6654
Epoch 5/10
154/154 [==============================] - 0s 1ms/step - loss: 0.6597 - accuracy: 0.6771
Epoch 6/10
154/154 [==============================] - 0s 1ms/step - loss: 0.6524 - accuracy: 0.6875
Epoch 7/10
154/154 [==============================] - 0s 1ms/step - loss: 0.6251 - accuracy: 0.6849
Epoch 8/10
154/154 [==============================] - 0s 1ms/step - loss: 0.6267 - accuracy: 0.6784
Epoch 9/10
154/154 [==============================] - 0s 1ms/step - loss: 0.6240 - accuracy: 0.7018
Epoch 10/10
154/154 [==============================] - 0s 1ms/step - loss: 0.6139 - accuracy: 0.6849

In [ ]:
#다중 분류일경우 활성화 함수 softmax 사용

In [134]:
df = pd.read_csv('iris3.csv')

In [135]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [136]:
X = df.iloc[:,0:4]
Y = df.iloc[:, 4]

In [137]:
y=pd.get_dummies(Y)    # 범주형 자료일 경우 무조건 원핫 인코딩 해줘야함 

In [138]:
y

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [139]:
model = Sequential()

In [140]:
model.add(Dense(16,  input_dim=4, activation='relu')) # input 은 열 갯수 차원만큼 설정 해줌
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [141]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [142]:
model.fit(X,y, epochs=40, batch_size=5)

Epoch 1/40
30/30 [==============================] - 0s 1ms/step - loss: 1.4147 - accuracy: 0.3333
Epoch 2/40
30/30 [==============================] - 0s 1ms/step - loss: 1.0829 - accuracy: 0.3800
Epoch 3/40
30/30 [==============================] - 0s 1ms/step - loss: 0.9771 - accuracy: 0.4733
Epoch 4/40
30/30 [==============================] - 0s 1ms/step - loss: 0.9461 - accuracy: 0.6067
Epoch 5/40
30/30 [==============================] - 0s 997us/step - loss: 0.9274 - accuracy: 0.5000
Epoch 6/40
30/30 [==============================] - 0s 1ms/step - loss: 0.8963 - accuracy: 0.5733
Epoch 7/40
30/30 [==============================] - 0s 1ms/step - loss: 0.8662 - accuracy: 0.5267
Epoch 8/40
30/30 [==============================] - 0s 996us/step - loss: 0.8410 - accuracy: 0.8133
Epoch 9/40
30/30 [==============================] - 0s 1ms/step - loss: 0.8105 - accuracy: 0.7400
Epoch 10/40
30/30 [==============================] - 0s 998us/step - loss: 0.7756 - accuracy: 0.8200
Epoch 11/40
3

In [ ]:
5.9	3.0	5.1	1.8	Iris-virginica

In [145]:
model.predict(np.array([[5.9,3.0,5.1,1.8]]))

1/1 [==============================] - 0s 42ms/step


array([[0.00123006, 0.23586336, 0.7629066 ]], dtype=float32)

In [144]:
import numpy as np

In [ ]:
배경 : 
전경 ㅣ 본안얼굴(사람 ), 강아지, 고양이, 다른사람, 자전거, 컴퓨터 ....